In [1]:
# PyCaret 라이브러리 설치
!pip install pycaret
!pip install markupsafe==2.0.1
!pip install catboost
!pip install pyod 

# 베이지안 탐색 라이브러리
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 320 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 63.8 MB/s 
     |████████████████████████████████| 125 kB 76.2 MB/s 
     |████████████████████████████████| 17.9 MB 424 kB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 6.8 MB 51.4 MB/s 
     |████████████████████████████████| 1.7 MB 60.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 2.0 MB 52.2 MB/s 
     |████████████████████████████████| 25.9 MB 2.6 MB/s 
     |████████████████████████████████| 88 kB 6.6 MB/s 
     |████████████████████████████████| 167 kB 65.1 MB/s 
     |████████████████████████████████| 10.4 MB 24.5 MB/s 
     |████████████████████████████████| 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 2.1.1
    Uninstalling MarkupSafe-2.1.1:
      Successfully uninstalled MarkupSafe-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires markupsafe~=2.1.1, but you have markupsafe 2.0.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |█

# 1. 환경 설정

## 1-1. 라이브러리

In [2]:
# from lightgbm import LGBMClassifier
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
# mport lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

## 1-2. 데이터셋 로드

In [22]:
from google.colab import drive
drive.mount('/content/drive') # 구글 드라이브를 사용하는 경우

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train_df = pd.read_csv("/content/drive/MyDrive/dacon/unsupervised_card/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/dacon/unsupervised_card/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/dacon/unsupervised_card/data/test.csv") 

print("================데이터셋 크기================")
print(train_df.shape) # 569 샘플, 32 컬럼으로 구성된 데이터
print("================데이터셋 구조================")
print(train_df.info())
print("================데이터셋 샘플================")
train_df.head() # 데이터 샘플

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
================데이터셋 크기================
(113842, 31)
================데이터셋 구조================
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113842 entries, 0 to 113841
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      113842 non-null  int64  
 1   V1      113842 non-null  float64
 2   V2      113842 non-null  float64
 3   V3      113842 non-null  float64
 4   V4      113842 non-null  float64
 5   V5      113842 non-null  float64
 6   V6      113842 non-null  float64
 7   V7      113842 non-null  float64
 8   V8      113842 non-null  float64
 9   V9      113842 non-null  float64
 10  V10     113842 non-null  float64
 11  V11     113842 non-null  float64
 12  V12     113842 non-null  float64
 13  V13     113842 non-null  float64
 14  V14     113842 non-null  float64
 15  V15     113842 n

,ID,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30
0,3,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,4.983721,-0.994972
1,4,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,1.418291,-0.994972
2,6,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,-0.256131,-0.994960
3,8,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,0.262698,-0.994901
4,9,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,0.994900,-0.994901


## 1-3. 데이터셋 지정
- val 데이터 사용 시만 실행

In [23]:
train_df = val_df.copy()
val_y = train_df['Class']
train_df = train_df.drop(columns=['Class'])

In [24]:
train_df.shape

(28462, 31)

# 2. 데이터 전처리

In [25]:
val_normal, val_fraud = val_df['Class'].value_counts()
val_contamination = val_fraud / val_normal
print(f'Validation contamination : [{val_contamination}]')

Validation contamination : [0.0010551491277433877]


In [26]:
# Train dataset은 Label이 존재하지 않음
train_x = train_df.drop(columns=['ID']) # Input Data

In [27]:
from pycaret.anomaly import *

setup_clf = setup(data=train_x,
                   normalize=True,
                   normalize_method ='robust',
                   transformation=True,
                   # transformationethod='quantile',
                   # pca=True,
                   remove_multicollinearity=True,
                   remove_perfect_collinearity=True,
                   session_id=777)

,Description,Value
0,session_id,777
1,Original Data,"(28462, 30)"
2,Missing Values,False
3,Numeric Features,30
4,Categorical Features,0
5,Ordinal Features,False
6,High Cardinality Features,False
7,High Cardinality Method,None
8,Transformed Data,"(28462, 30)"
9,CPU Jobs,-1


# 3. 모델링

## 3-1. 전체 모델 비교

In [28]:
models()

,Name,Reference
ID,,
abod,Angle-base Outlier Detection,pyod.models.abod.ABOD
cluster,Clustering-Based Local Outlier,pyod.models.cblof.CBLOF
cof,Connectivity-Based Local Outlier,pyod.models.cof.COF
iforest,Isolation Forest,pyod.models.iforest.IForest
histogram,Histogram-based Outlier Detection,pyod.models.hbos.HBOS
knn,K-Nearest Neighbors Detector,pyod.models.knn.KNN
lof,Local Outlier Factor,pyod.models.lof.LOF
svm,One-class SVM detector,pyod.models.ocsvm.OCSVM
pca,Principal Component Analysis,pyod.models.pca.PCA


In [29]:
models_list = models().index.to_list()
models_list.remove('cof') # cof는 RAM 메모리 초과로 제외
models_list.remove('sos') # sos는 RAM 메모리 초과로 제외
models_list

['abod',
 'cluster',
 'iforest',
 'histogram',
 'knn',
 'lof',
 'svm',
 'pca',
 'mcd',
 'sod']

In [30]:
f1_list = []

for m in models_list:
    model = create_model(m,fraction=val_contamination)
    print(model)

    result = assign_model(model)
    # result.head()

    print(result['Anomaly_Score'].describe())
    print(f1_score(val_y, result['Anomaly']))
    f1_list.append([m, f1_score(val_y, result['Anomaly'])])
    print(print(classification_report(val_y, result['Anomaly'])))

SOD(alpha=None, contamination=0.0010551491277433877, n_neighbors=None,
  ref_set=None)
count    2.846200e+04
mean     2.577987e-01
std      2.153707e-01
min      5.051927e-08
25%      1.086646e-01
50%      2.079120e-01
75%      3.430494e-01
max      4.333238e+00
Name: Anomaly_Score, dtype: float64
0.22950819672131148
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.23      0.23      0.23        30

    accuracy                           1.00     28462
   macro avg       0.61      0.62      0.61     28462
weighted avg       1.00      1.00      1.00     28462

None


In [31]:
f1_list.sort(key = lambda x:x[1], reverse=True)
f1_list

[['iforest', 0.39344262295081966],
 ['histogram', 0.39344262295081966],
 ['cluster', 0.36065573770491804],
 ['pca', 0.36065573770491804],
 ['knn', 0.29508196721311475],
 ['sod', 0.22950819672131148],
 ['lof', 0.13114754098360656],
 ['svm', 0.13114754098360656],
 ['abod', 0.09836065573770492],
 ['mcd', 0.06557377049180328]]

## 3-2. 최적 모델로 모델링
- [histogram reference](https://pyod.readthedocs.io/en/latest/pyod.models.html)

In [32]:
model = create_model('histogram',fraction=val_contamination)
print(model)

HBOS(alpha=0.1, contamination=0.0010551491277433877, n_bins=10, tol=0.5)


In [33]:
# 하이퍼 파라미터 설정
model.n_bins = 10
model.tol = 0.01
model.alpha = 0.01

In [34]:
model = create_model(model,fraction=val_contamination)
print(model)

HBOS(alpha=0.01, contamination=0.0010551491277433877, n_bins=10, tol=0.01)


In [35]:
result = assign_model(model)

# 4. 성능평가

In [36]:
print(result['Anomaly_Score'].describe())
print(f1_score(val_y, result['Anomaly']))
print(print(classification_report(val_y, result['Anomaly'])))

count    28462.000000
mean        65.097193
std          8.525999
min         51.688776
25%         59.584966
50%         63.197114
75%         68.317378
max        158.568905
Name: Anomaly_Score, dtype: float64
0.4262295081967213
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.42      0.43      0.43        30

    accuracy                           1.00     28462
   macro avg       0.71      0.72      0.71     28462
weighted avg       1.00      1.00      1.00     28462

None


# 5. 제출 파일 생성

In [15]:
test_x = test_df.drop(columns=['ID'])

In [17]:
test_pred = model.predict(test_x) # model prediction


In [19]:
submit = pd.read_csv('/content/drive/MyDrive/dacon/unsupervised_card/data/sample_submission.csv')
submit.head()

,ID,Class
0,AAAA0x1,1
1,AAAA0x2,1
2,AAAA0x5,1
3,AAAA0x7,1
4,AAAA0xc,1


In [20]:
submit['Class'] = test_pred
submit.to_csv('/content/drive/MyDrive/dacon/unsupervised_card/submit_histogram.csv', index=False)

In [21]:
submit['Class'].value_counts()

0    142334
1       169
Name: Class, dtype: int64